# <u>S</u>tabilizer <u>S</u>ub<u>S</u>pace (S3) Projections

A library providing the necessary functionality to perform stabilizer subspace projections over systems of Pauli 
operators.

This facilitates implementations of qubit reduction techniques such as [*tapering*](https://arxiv.org/abs/1701.08213) and [*Contextual-Subspace VQE*](https://doi.org/10.22331/q-2021-05-14-456), both of which are demonstrated here.

In [32]:
from qondense.tapering import tapering
from qondense.cs_vqe import cs_vqe
import qondense.utils.qonversion_tools as qonvert
from qondense.utils.operator_toolkit import exact_gs_energy, plot_ground_state_amplitudes

import numpy as np
import openfermion as of
import openfermionpyscf as ofpyscf
from itertools import combinations
import json

First of all, we construct our molecular Hamiltonian:

In [4]:
with open('data/molecule_geometries/molecule_data.json') as jfile:
    molecule_geometries = json.load(jfile)
print(molecule_geometries.keys())

dict_keys(['H2_3-21G_SINGLET', 'H6_STO-3G_SINGLET', 'H2_6-31G_SINGLET', 'H2_6-311G_SINGLET', 'H3+_STO-3G_SINGLET', 'H3+_3-21G_SINGLET', 'HeH+_3-21G_SINGLET', 'HeH+_6-311G_SINGLET', 'H2O_STO-3G_SINGLET', 'BeH+_STO-3G_SINGLET', 'LiH_STO-3G_SINGLET', 'CH+_STO-3G_SINGLET', 'HF_STO-3G_SINGLET', 'B+_STO-3G_SINGLET', 'B_STO-3G_DOUBLET', 'N_STO-3G_QUARTET', 'OH-_STO-3G_SINGLET', 'O_STO-3G_TRIPLET', 'CH2_STO-3G_TRIPLET', 'BeH2_STO-3G_SINGLET', 'Be_STO-3G_SINGLET', 'C_STO-3G_TRIPLET', 'NH_STO-3G_SINGLET', 'Ne_STO-3G_SINGLET', 'F_STO-3G_DOUBLET', 'Li_STO-3G_DOUBLET', 'BH_STO-3G_SINGLET', 'NeH+_STO-3G_SINGLET', 'NH2+_STO-3G_SINGLET', 'BH2+_STO-3G_SINGLET', 'HCl_STO-3G_SINGLET', 'H4_STO-3G_SINGLET', 'NH3_STO-3G_SINGLET', 'F2_STO-3G_SINGLET', 'HCN_STO-3G_SINGLET', 'CH4_STO-3G_SINGLET', 'CH3OH_STO-3G_SINGLET', 'C2H6_STO-3G_SINGLET', 'CH3CN_STO-3G_SINGLET', 'CH3CHO_STO-3G_SINGLET', 'CH3CHOHCH3_STO-3G_SINGLET', 'CHONH2_STO-3G_SINGLET', 'CO2_STO-3G_SINGLET', 'O2_STO-3G_SINGLET', 'O3_STO-3G_SINGLET', 'HO

In [22]:
from qiskit_nature.operators.second_quantization.fermionic_op import FermionicOp
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver, MethodType
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit_nature.circuit.library.ansatzes.ucc import UCC
from qiskit_nature.circuit.library.initial_states.hartree_fock import hartree_fock_bitstring
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.opflow.primitive_ops.tapered_pauli_sum_op import TaperedPauliSumOp

speciesname = 'H2O_STO-3G_SINGLET'
mol_data = molecule_geometries[speciesname]
if 'name' in mol_data:
    print(mol_data['name'])
    
atoms = mol_data['atoms']
coords = mol_data['coords']
basis = mol_data['basis']
multiplicity = mol_data['multiplicity']
charge = mol_data['charge']
geometry = list(zip(atoms, coords))

geometry = []
for index, a in enumerate(atoms):
    geometry.append((a, coords[index]))

# construct electronic structure problem in Qiskit Nature for Z2 symmetry identification
molecule_qiskit = Molecule(geometry=geometry, charge=charge, multiplicity=multiplicity) 
driver = ElectronicStructureMoleculeDriver(molecule_qiskit, basis=basis, 
                                            method = MethodType.RHF, 
                                            driver_type=ElectronicStructureDriverType.PYSCF)
es_problem = ElectronicStructureProblem(driver)
ham_2ndQ   = es_problem.second_q_ops()[0]
num_qubits = ham_2ndQ.register_length
print(num_qubits)

# Determine tapering parameters
qubit_converter = QubitConverter(JordanWignerMapper(), z2symmetry_reduction='auto')
ham_ref    = qubit_converter.convert(ham_2ndQ)
ham_dict   = qonvert.PauliOp_to_dict(ham_ref)
true_gs = exact_gs_energy(ham_dict)[0]

14


Perform qubit tapering to reduce the number of necessary qubits:

In [28]:
alpha_beta_nums = list(es_problem.num_particles)
if charge==-1:
    alpha_beta_nums[1]=alpha_beta_nums[1]-1
if charge==+1:
    alpha_beta_nums[0]=alpha_beta_nums[0]+1
    
def hartree_fock_state(taper=False, hf_bool=False):
    hf_config_bool = hartree_fock_bitstring(num_particles=alpha_beta_nums,
                                            num_spin_orbitals=num_qubits)
    if not hf_bool:
        hf_config = ''.join([str(int(b)) for b in hf_config_bool])[::-1]
        return hf_config
    else:
        return hf_config_bool
    
hf_state = hartree_fock_state()
hf_state, true_gs

('00111110011111', -84.2498665660213)

In [30]:
taper_hamiltonian = tapering(hamiltonian=ham_dict, 
                               ref_state=hf_state)
taper_hamiltonian.update_S3_projection([-1,1,1,-1])
ham_tap = taper_hamiltonian.taper_it()
tap_gs_energy, tap_gs = exact_gs_energy(ham_tap._dict())
print(taper_hamiltonian.symmetry_sec)
print(f'In the sector {taper_hamiltonian.symmetry_sec}, we find the ground state energy to be {tap_gs_energy}.')
print(f'The absolute error is {tap_gs_energy-true_gs}.')

[1, 1, 1, 1]
In the sector [1, 1, 1, 1], we find the ground state energy to be -84.24986656602107.
The absolute error is 2.2737367544323206e-13.


Build CS-VQE model with legacy code for comparison:

In [70]:
from qondense.utils.cs_vqe_tools_legacy import csvqe_approximations_heuristic, quasi_model, greedy_dfs, find_gs_noncon

with open('data/model_data/model_data.json', 'r') as infile:
    model_data = json.load(infile)
with open('data/model_data/HCl_STO-3G_SINGLET_model_data.json', 'r') as infile:
    model_data['HCl_STO-3G_SINGLET'] = json.load(infile)['HCl_STO-3G_SINGLET']
with open('data/model_data/F2_STO-3G_SINGLET_model_data.json', 'r') as infile:
    model_data['F2_STO-3G_SINGLET'] = json.load(infile)['F2_STO-3G_SINGLET']
    
mols = ["Be_STO-3G_SINGLET",
       "B_STO-3G_DOUBLET",
       "LiH_STO-3G_SINGLET",
       "BeH+_STO-3G_SINGLET",
       "HF_STO-3G_SINGLET",
       "BeH2_STO-3G_SINGLET",
       "H2O_STO-3G_SINGLET",
       "HCl_STO-3G_SINGLET",
       "F2_STO-3G_SINGLET"]

noncon_data = {}
for speciesname in mols:
    ham = model_data[speciesname]['ham']
    terms_noncon = model_data[speciesname]['terms_noncon']
    n_qubits = model_data[speciesname]['num_qubits']
    ham_noncon = {op:ham[op] for op in terms_noncon}
    true_gs = model_data[speciesname]['truegs']
    
    noncon_stuff = find_gs_noncon(ham_noncon=ham_noncon)
    G_val, A_val = noncon_stuff[1]
    G, A = noncon_stuff[2][:2]
    noncon_data[speciesname]={'G':G, 'nu':G_val, 'A':A, 'r':A_val}

noncon_data

{'Be_STO-3G_SINGLET': {'G': ['ZZZIZ', 'IZIIZ', 'IIZIZ', 'IIIIZ'],
  'nu': [-1, -1, -1, -1],
  'A': ['IIIXZ', 'ZZZZI'],
  'r': [1.722957938005497e-08, -0.9999999999999999]},
 'B_STO-3G_DOUBLET': {'G': ['ZZZIZ', 'IZIII', 'IIZIZ', 'IIIIZ'],
  'nu': [1, 1, 1, -1],
  'A': ['ZZZZI', 'IIIXZ'],
  'r': [1.0, 1.633162674519949e-11]},
 'LiH_STO-3G_SINGLET': {'G': ['ZIIIIIII',
   'IZZIIZZZ',
   'IIZIIZZZ',
   'IIIIZIII',
   'IIIIIZZZ',
   'IIIIIIZZ',
   'IIIIIIIZ'],
  'nu': [1, -1, -1, -1, -1, -1, -1],
  'A': ['ZIIXIZZZ', 'ZZZZZIII'],
  'r': [-6.022245751909695e-09, -1.0]},
 'BeH+_STO-3G_SINGLET': {'G': ['ZIIIIIII',
   'IZZIIZZZ',
   'IIZIIZZZ',
   'IIIIZIII',
   'IIIIIZZZ',
   'IIIIIIZZ',
   'IIIIIIIZ'],
  'nu': [1, 1, 1, -1, 1, 1, -1],
  'A': ['ZIIXIZZZ', 'ZZZZZIII'],
  'r': [0.002132853189680987, -0.9999977254660489]},
 'HF_STO-3G_SINGLET': {'G': ['ZZZZZIII',
   'IZZIIZZZ',
   'IIZIIZZZ',
   'IIIIZIII',
   'IIIIIZZZ',
   'IIIIIIZI',
   'IIIIIIIZ'],
  'nu': [1, 1, -1, -1, 1, -1, -1],
  'A': ['XZ

In [57]:
N=8
set(range(N)) - set([N-1-i for i in [0 ,1 ,3 ,7]])

{1, 2, 3, 5}

In [58]:
ops = ['IZZIIZZZ',
   'IIZIIZZZ',
      'IIIIIZZZ']

N-len(ops)

5

In [59]:
def to_sigma(P):
    return ' '.join(['\sigma_{%i}^{(%i)}'%(q_map[Pi], i) for i,Pi in enumerate(P[::-1]) if Pi!='I'][::-1])

print(', '.join([to_sigma(P) for P in ops])+', C(\\bm{r})')

\sigma_{3}^{(6)} \sigma_{3}^{(5)} \sigma_{3}^{(2)} \sigma_{3}^{(1)} \sigma_{3}^{(0)}, \sigma_{3}^{(5)} \sigma_{3}^{(2)} \sigma_{3}^{(1)} \sigma_{3}^{(0)}, \sigma_{3}^{(2)} \sigma_{3}^{(1)} \sigma_{3}^{(0)}, C(\bm{r})


In [17]:
sorted([16-1-i for i in [0, 9, 3, 12, 7, 15, 14, 6, 11, 2]])

[0, 1, 3, 4, 6, 8, 9, 12, 13, 15]

In [18]:
q_map = {'I':0, 'X':1, 'Y':2, 'Z':3}
def to_sigma(P):
    return ' '.join(['\sigma_{%i}^{(%i)}'%(q_map[Pi], i) for i,Pi in enumerate(P[::-1]) if Pi!='I'][::-1])

for speciesname, params in noncon_data.items():
    print(speciesname)
    
    n_qubits = model_data[speciesname]['chem_acc_num_q']
    G = model_data[speciesname]['symmetrygen']
    nu = model_data[speciesname]['tapersector']
    
    sigmas = []
    for Gi in G:
        sigmas.append('$ '+to_sigma(Gi)+' $')
    
    print('\\multicolumn{2}{|c|}{\\textbf{Tapering parameters}} \\\ \\hline\\hline')
    print('\\textbf{Symmetry generators} $\\mathcal{G}$ &')
    print(', '.join(sigmas) +' \\\ \hline')
    print('\\textbf{Sector} $\\bm{\\nu}$ & \\{'+str(nu)[1:-1]+'\\} \\\ \\hline\\hline')
    print()
    
    G = params['G']
    nu = params['nu']
    A = params['A']
    r = params['r']
    
    sigmas = []
    for Gi in G:
        sigmas.append('$ '+to_sigma(Gi)+' $')
    
    rC = ' + '.join([f'{ri} \\cdot {to_sigma(Ai)}' for ri,Ai in zip(r,A)])
    
    print('\\multicolumn{2}{|c|}{\\textbf{Noncontextual model}} \\\ \\hline\\hline')
    print('\\textbf{Symmetry generators} $\\mathcal{G}$ &')
    print(',\\; '.join(sigmas) +' \\\ \hline')
    print('\\textbf{Sector} $\\bm{\\nu}$ & \\{'+str(nu)[1:-1]+'\\} \\\ \\hline')
    print('\\textbf{Clique observable} $C(\\bm{r})$ & $'+rC+'$ \\\ \\hline\\hline')
    print('\\multicolumn{2}{|c|}{\\textbf{'+str(n_qubits)+'-qubit CS-VQE parameters}} \\\ \\hline\\hline')
    print()

NameError: name 'noncon_data' is not defined

# Contextual-Subspace VQE
Here we run through the basic CS-VQE functionality...

When the `cs_vqe` class is initiated it generates a set of stabilizers defined through the CS-VQE method. These stabilizers are consistent with the noncontextual ground state energy, a classical estimate of the true value that is *at least as accurate* as Hartree-Fock.

In [37]:
from qondense.utils.cs_vqe_tools_legacy import csvqe_approximations_heuristic, quasi_model, greedy_dfs
ham = ham_tap._dict()
terms_noncon = greedy_dfs(ham, cutoff=10)[-1]
hf_tapered = taper_hamiltonian.taper_ref_state()

In [42]:
ham

{'IIIIIIIIII': (-55.420762162886454+0j),
 'XIIIIIIIII': (0.05791287308384965+0j),
 'XXIIIIIIII': (0.0075698414451820925+0j),
 'XIXIIIIIII': (0.003167351336415194+0j),
 'IXXIIIIIII': (-0.01593846664015542+0j),
 'IIIXIIIIII': (-0.004387061336512375+0j),
 'XXIXIIIIII': (0.001166599161044743+0j),
 'IIXXIIIIII': (0.12506426575286844+0j),
 'IXXXIIIIII': (0.0076799647559147605+0j),
 'IIIIXIIIII': (-0.028906946835371095+0j),
 'XXIIXIIIII': (0.011767712432203155+0j),
 'IXXIXIIIII': (0.024801861314879207+0j),
 'IIIXXIIIII': (0.003081688165841763+0j),
 'IIIIXXIIII': (0.009691499373174038+0j),
 'XIIIXXIIII': (0.0005053446912447325+0j),
 'IIIIXIXIII': (-0.05791287308384965+0j),
 'XIIIXIXIII': (-0.016861566711241116+0j),
 'IIIIIXXIII': (-0.0005053446912447325+0j),
 'XIIIIXXIII': (-0.014023611932222211+0j),
 'IIIIIIXXII': (-0.011767712432203155+0j),
 'XXIIIIXXII': (0.011729648031149387+0j),
 'IXXIIIXXII': (0.004923473006100793+0j),
 'IIIXIIXXII': (-0.0012470236322305755+0j),
 'IIIIXIXXII': (-0.007569

In [38]:
cs = cs_vqe(ham, noncontextual_set=terms_noncon, ref_state=hf_tapered)

#match_original = abs(cs.ngs_energy-gs_noncon[0])<1e-13
print("Noncontextual GS energy:", cs.ngs_energy)#, ' // matches original?', match_original)
print("Symmetry generators:    ", cs.generators)
print("Clique representatives: ", cs.cliquereps)
print("Generator eigenvalues:  ", cs.nu)
print("Clique operator coeffs: ", cs.r)

Noncontextual GS energy: (-12.647109365792382+0j)
Symmetry generators:     ['IIIIIIIZII', 'ZIIIIIIIII', 'IZIIIIIIII', 'IIZIIIIIII', 'IIIZIIIIII', 'IIIIZZZIZZ', 'IIIIIZIIII', 'IIIIIIZIII', 'IIIIIIIIZI', 'IIIIIIIIIZ']
Clique representatives:  ['IIIIZZZXII', 'IIIIIIIZII']
Generator eigenvalues:   [1 1 1 1 1 1 1 1 1 1]
Clique operator coeffs:  [ 0.97226119-0.j -0.23389781-0.j]


/home/tweaving/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:4712: ComplexWarning: Casting complex values to real discards the imaginary part
  values = array(values, copy=False, ndmin=arr.ndim, dtype=arr.dtype)


In [38]:
exact_gs_energy(cs.ham_noncontextual._dict)

(-75.09275105309187,
 array([-1.90572374e-17-3.34845061e-17j,  2.88114674e-17-1.26256488e-16j,
         1.74195990e-17+5.38776746e-17j, ...,
         3.02132971e-15-2.82873557e-15j,  3.44409390e-18-5.88601988e-18j,
        -3.27511327e-16+1.25849869e-17j]))

In [31]:
chemaccnumq = list(np.array(csvqe[2])<0.0016).index(True) #mol['chem_acc_num_q']

#exact, gs = exact_gs_energy(cs.hamiltonian)
print('Exact energy:',true_gs)
print('Noncon error:', cs.ngs_energy-true_gs)
print(f'Target_error for {chemaccnumq} qubits:', csvqe[2][chemaccnumq])# exact_gs_energy(mol['ham_reduced'][num_sim_q])[0]-exact)

Exact energy: -75.0108466481708
Noncon error: -0.08190440492062123
Target_error for 3 qubits: 0.0003265059338311005


**CS-VQE is sensitive to the choice of stabilizers we wish to enforce.**

Below, we drop stabilizer constraints iteratively, choosing that which minimizes the energy at each step.

In [34]:
stab_index_pool = list(range(len(cs.generators)))

optimal_errors = {}
for num_sim_q in range(2,cs.n_qubits):
    cs_vqe_errors = []
    for order in combinations(stab_index_pool, cs.n_qubits - num_sim_q):
        order = list(order)
        ham_cs = cs.contextual_subspace_hamiltonian(stabilizer_indices=order)#list(range(cs_vqe_mol.num_qubits)),
                                                                    #projection_qubits=order)
        cs_energy, cs_vector = exact_gs_energy(ham_cs)
        cs_vqe_errors.append((cs_energy-true_gs, order))
        
    cs_vqe_errors = sorted(cs_vqe_errors, key=lambda x:x[0])
    error, stab_index_pool = cs_vqe_errors[0]
    
    optimal_errors[num_sim_q]={}
    optimal_errors[num_sim_q]['error'] = error
    optimal_errors[num_sim_q]['stab_indices'] = list(stab_index_pool)
    
for num_sim_q in optimal_errors:
    error = optimal_errors[num_sim_q]['error']
    #diff_will = error-csvqe[2][num_sim_q]
    #print(diff_will)
    stab_indices = optimal_errors[num_sim_q]['stab_indices']
    print(f'Performing {num_sim_q}-qubit CS-VQE, we may obtain',
          f'an absolute error of {error:.8f},\n',
          f'enforcing the stabilizers {stab_indices}, {[cs.generators[i] for i in stab_indices]}\n'
         )

Performing 2-qubit CS-VQE, we may obtain an absolute error of 0.04753275,
 enforcing the stabilizers [1, 2, 3, 4, 5, 6, 7, 9], ['ZIZZIZZIIZ', 'IZZIZZIZIZ', 'IIZZIZZIIZ', 'IIIZZIZZII', 'IIIIZZIZIZ', 'IIIIIZZIIZ', 'IIIIIIZZII', 'IIIIIIIIIZ']

Performing 3-qubit CS-VQE, we may obtain an absolute error of 0.04342844,
 enforcing the stabilizers [1, 2, 3, 4, 5, 6, 9], ['ZIZZIZZIIZ', 'IZZIZZIZIZ', 'IIZZIZZIIZ', 'IIIZZIZZII', 'IIIIZZIZIZ', 'IIIIIZZIIZ', 'IIIIIIIIIZ']

Performing 4-qubit CS-VQE, we may obtain an absolute error of 0.03998331,
 enforcing the stabilizers [1, 2, 3, 4, 5, 9], ['ZIZZIZZIIZ', 'IZZIZZIZIZ', 'IIZZIZZIIZ', 'IIIZZIZZII', 'IIIIZZIZIZ', 'IIIIIIIIIZ']

Performing 5-qubit CS-VQE, we may obtain an absolute error of 0.03531261,
 enforcing the stabilizers [1, 2, 3, 4, 9], ['ZIZZIZZIIZ', 'IZZIZZIZIZ', 'IIZZIZZIIZ', 'IIIZZIZZII', 'IIIIIIIIIZ']

Performing 6-qubit CS-VQE, we may obtain an absolute error of 0.01577331,
 enforcing the stabilizers [1, 2, 3, 4], ['ZIZZIZZIIZ', 'IZZIZZI

Suppose we have access to just 3 qubits on some quantum device... then we may construct the corresponding 3-qubit CS-VQE model, obtaining the reduced Hamiltonian, Ansatz operator and noncontextual reference state.

In [9]:
num_sim_q = 4
stab_indices = [0,1,2,3]# optimal_errors[num_sim_q]['stab_indices']
ham_cs = cs.contextual_subspace_hamiltonian(stabilizer_indices=stab_indices)
#anz_cs = cs._contextual_subspace_projection(operator=ansatz,stabilizer_indices=stab_indices)
#ngs = cs.noncontextual_ground_state(stabilizer_indices=stab_indices)

In [10]:
print('Reduced Hamiltonian:\n', ham_cs)
print('\nReduced Ansatz:\n', anz_cs)
print('\nReference state:', ngs)

Reduced Hamiltonian:
 {'I': -12.68137833, 'Z': -1.67050214}


NameError: name 'anz_cs' is not defined

In [ ]:
plot_ground_state_amplitudes(operator=ham_cs, num_qubits=num_sim_q)#, reverse_bitstrings=True)

## Running CS-VQE

Finally, we may perform a VQE routine taking as input the reduced Hamiltonian and Ansatz defined above.

In [ ]:
import warnings; warnings.filterwarnings("ignore", category=DeprecationWarning)
import qreduce.utils.circuit_tools as circ
import qreduce.utils.circuit_execution_tools as circ_ex
from qiskit.circuit import QuantumCircuit

In [ ]:
qc = QuantumCircuit(num_sim_q)
for index, bit in enumerate(ngs):
    q_pos = num_sim_q-1-index
    if int(bit)==1:
        qc.x(q_pos)

qc = circ.circ_from_paulis(circ=qc, paulis=list(anz_cs.keys()), trot_order=2)
circ.cancel_pairs(circ=qc, hit_set={'s', 'sdg'})
circ.cancel_pairs(circ=qc, hit_set={'h', 'h'})
bounds = np.array([(a-np.pi/2, a+np.pi/2) for a in anz_cs.values()]) # optimization bounds
qc.parameter_bounds = bounds

qc.draw(output='mpl')

In [ ]:
vqe_result = circ_ex.vqe_simulation(ansatz=qc, operator=ham_cs, init_params=np.array(list(anz_cs.values())))

In [ ]:
log_errors = np.log10(np.square(np.array(vqe_result['values'])-exact))
plt.plot(log_errors, color='black', label='Optimizer output')
plt.hlines(np.log10(0.0016**2), 0, vqe_result['counts'][-1], label='Chemical accuracy', color='green')
plt.legend()

In [ ]:
np.array([0,0]).dot(np.array([1,2]))

In [48]:
print(exact_gs_energy({'II':1, 'XX':1, 'YY':1, 'ZZ':1, 'ZX':1})[0])
print(exact_gs_energy({'XX':1, 'YY':1, 'ZZ':1})[0])

-2.236067977499789
-3.0
